# This notebook explores the results for the agent based model

In [1]:
import re
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', None )

# Loading localidades GeoJSON for reference

# Loading ABM results

In [3]:
city = 'cali'
scenario = 'no-cuarentena'
init_date = pd.to_datetime( '03/06/2020' )

In [4]:
path = './data/agents/{}/{}/*.csv'.format( city, scenario )

In [5]:
files = glob.glob( path )

In [6]:
files.sort()

In [7]:
agents = []
exp = 0

for file in files:
    df = pd.read_csv( file )
    df[ 'Fecha' ] = df[ 'Día' ].apply( lambda x: init_date + pd.DateOffset( days = x ) )
    df[ 'exp' ] = exp
    agents.append( df )
    exp = exp + 1

agents = pd.concat( agents )
del agents[ 'Unnamed: 0' ]

In [8]:
agents.shape

(9600, 44)

In [9]:
agents.dtypes

Susceptibles                                   int64
Total infectados                               int64
Graves                                         int64
Críticos                                       int64
Leves                                          int64
Recuperados                                    int64
Rt                                           float64
Recuento_zonas                                object
0-9                                           object
10-19                                         object
20-39                                         object
40-59                                         object
>60                                           object
En_cuarentena                                 object
Restringido_movilidad                         object
Vivos                                          int64
Cuarentena_permanente                         object
Día                                            int64
Contactos_prom_trabajo                       f

In [10]:
agents.tail()

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Recuento_zonas,0-9,10-19,20-39,40-59,>60,En_cuarentena,Restringido_movilidad,Vivos,Cuarentena_permanente,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Contactos_prom_trabajo_conm,Contactos_prom_transporte_conm,Contactos_prom_casa_conm,Nuevos_infectados,Detectados,En_testing,En_cama,En_UCI,Detectados_por_intervencion,#Intervenidos,porcentaje_edad_activa_por_zona,porcentaje_edad_activa,tasa_letalidad_por_zona,tasa_letalidad,total_muertos,total_muertos_por_zona,total_muertos_edad_activa,total_muertos_edad_activa_por_zona,zonas intervenidas,Graves_efectivos,Criticos_efectivos,Fecha,exp
295,21831,0,0,0,0,15486,0.0,"{18: [1958, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2145, ...","[2596, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2522, ...","[7211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4880, ...","[5900, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3723, ...","[4058, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2216, ...","[0, 0, 0, 0, 0]","[6240, 0, 0, 0, 4350]",37317,"[0, 0, 0, 0, 0]",295,0.653429,1.801109,1.610472,0.912336,2.514760,1.601751,0,387,0,49,4,0,0,"{1: 0.0, 2: 0.0009199632014719411, 3: 0.0, 4: ...",0.005939,"{1: 0.0, 2: 0.3333333333333333, 3: 0, 4: 0.375...",0.141907,64,"{1: 0, 2: 2, 3: 0, 4: 3, 5: 7, 6: 7, 7: 4, 8: ...",9,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: ...","{8: [[109, 15], [110, 15], [111, 15], [112, 15...",0,0,2020-12-26,31
296,21831,0,0,0,0,15486,0.0,"{18: [1958, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2145, ...","[2596, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2522, ...","[7211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4880, ...","[5900, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3723, ...","[4058, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2216, ...","[0, 0, 0, 0, 0]","[5028, 0, 0, 0, 3545]",37317,"[0, 0, 0, 0, 0]",296,0.751186,2.132460,1.620521,0.975230,2.768473,1.627644,0,387,0,49,4,0,0,"{1: 0.0, 2: 0.0009199632014719411, 3: 0.0, 4: ...",0.005939,"{1: 0.0, 2: 0.3333333333333333, 3: 0, 4: 0.375...",0.141907,64,"{1: 0, 2: 2, 3: 0, 4: 3, 5: 7, 6: 7, 7: 4, 8: ...",9,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: ...","{8: [[109, 15], [110, 15], [111, 15], [112, 15...",0,0,2020-12-27,31
297,21831,0,0,0,0,15486,0.0,"{18: [1958, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2145, ...","[2596, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2522, ...","[7211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4880, ...","[5900, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3723, ...","[4058, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2216, ...","[0, 0, 0, 0, 0]","[5975, 0, 0, 0, 4329]",37317,"[0, 0, 0, 0, 0]",297,0.654152,1.850068,1.594072,0.903676,2.555769,1.592678,0,387,0,49,4,0,0,"{1: 0.0, 2: 0.0009199632014719411, 3: 0.0, 4: ...",0.005939,"{1: 0.0, 2: 0.3333333333333333, 3: 0, 4: 0.375...",0.141907,64,"{1: 0, 2: 2, 3: 0, 4: 3, 5: 7, 6: 7, 7: 4, 8: ...",9,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: ...","{8: [[109, 15], [110, 15], [111, 15], [112, 15...",0,0,2020-12-28,31
298,21831,0,0,0,0,15486,0.0,"{18: [1958, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2145, ...","[2596, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2522, ...","[7211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4880, ...","[5900, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3723, ...","[4058, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2216, ...","[0, 0, 0, 0, 0]","[6531, 0, 0, 0, 4609]",37317,"[0, 0, 0, 0, 0]",298,0.629365,1.754562,1.604577,0.897200,2.501242,1.610498,0,387,0,49,4,0,0,"{1: 0.0, 2: 0.0009199632014719411, 3: 0.0, 4: ...",0.005939,"{1: 0.0, 2: 0.3333333333333333, 3: 0, 4: 0.375...",0.141907,64,"{1: 0, 2: 2, 3: 0, 4: 3, 5: 7, 6: 7, 7: 4, 8: ...",9,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: ...","{8: [[109, 15], [110, 15], [111, 15], [112, 15...",0,0,2020-12-29,31
299,21831,0,0,0,0,15486,0.0,"{18: [1958, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8...","[2066, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2145, ...","[2596, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2522, ...","[72

In [11]:
def sum_quarantine( x ):
    x = json.loads( x )
    people_in_q = x[ 0 ] + x[ 1 ] + x[ 2 ] + x[ 3 ] + x[ 4 ]
    return people_in_q

In [12]:
agents_norm = []

for exp in agents[ 'exp' ].unique():
    df = agents.loc[ agents[ 'exp' ] == exp ]
    total_people = df[ 'Vivos' ].values[ 0 ]
    df[ 'Graves' ] = df[ 'Graves' ] / total_people
    df[ 'Críticos' ] = df[ 'Críticos' ] / total_people
    df[ 'Fallecidos' ] = ( total_people - df[ 'Vivos' ] ) / total_people
    df[ 'R0' ] = ( df[ 'Nuevos_infectados' ].cumsum() / total_people )
    df[ 'Cuarentena' ] = df[ 'En_cuarentena' ].apply( lambda x: sum_quarantine( x ) ) / total_people
    agents_norm.append( df[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ] )

agents_norm = pd.concat( agents_norm )

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [13]:
agents_norm.tail()

,exp,Fecha,Graves,Críticos,Fallecidos,R0,Rt,Cuarentena
295,31,2020-12-26,0.0,0.0,0.001712,0.414515,0.0,0.0
296,31,2020-12-27,0.0,0.0,0.001712,0.414515,0.0,0.0
297,31,2020-12-28,0.0,0.0,0.001712,0.414515,0.0,0.0
298,31,2020-12-29,0.0,0.0,0.001712,0.414515,0.0,0.0
299,31,2020-12-30,0.0,0.0,0.001712,0.414515,0.0,0.0


In [14]:
scenario_name = 'Sin cuarentena'

var = 'Críticos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}-{}-{}.csv'.format( city, scenario, 'ucis' ), index = False )

var = 'Fallecidos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}-{}-{}.csv'.format( city, scenario, 'deaths' ), index = False )

var = 'R0'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}-{}-{}.csv'.format( city, scenario, 'r0' ), index = False )

var = 'Rt'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}-{}-{}.csv'.format( city, scenario, 'rt' ), index = False )

var = 'Cuarentena'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}-{}-{}.csv'.format( city, scenario, 'quarantine' ), index = False )

In [ ]:
def ci( x ):
    try:
        return 1.96 * np.std( x ) / np.sqrt( len( x ) )
    except ZeroDivisionError:
        return 0

agents_norm_grouped = agents_norm.groupby( 'Fecha' ).agg( { 'Graves': [ 'mean', ci ], 'Críticos': [ 'mean', ci ], 'Fallecidos': [ 'mean', ci ], 'R0': [ 'mean', ci ], 'Rt': [ 'mean', ci ], 'Cuarentena': [ 'mean' ] } )
agents_norm_grouped.columns = [ '_'.join( col ).strip() for col in agents_norm_grouped.columns.values ]

In [ ]:
agents_norm_grouped.tail()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Graves_mean' ], label = 'Graves', color = '#317372' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Graves_mean' ] - agents_norm_grouped[ 'Graves_ci' ] ), ( agents_norm_grouped[ 'Graves_mean' ] + agents_norm_grouped[ 'Graves_ci' ] ), color = '#317372', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Críticos_mean' ], label = 'Críticos', color = '#f98a4b' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Críticos_mean' ] - agents_norm_grouped[ 'Críticos_ci' ] ), ( agents_norm_grouped[ 'Críticos_mean' ] + agents_norm_grouped[ 'Críticos_ci' ] ), color = '#f98a4b', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Fallecidos_mean' ], label = 'Fallecidos', color = '#f64438' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Fallecidos_mean' ] - agents_norm_grouped[ 'Fallecidos_ci' ] ), ( agents_norm_grouped[ 'Fallecidos_mean' ] + agents_norm_grouped[ 'Fallecidos_ci' ] ), color = '#f64438', alpha = .1 )

plt.title( 'Graves / Críticos / Fallecidos' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Casos' )
plt.grid( True )
plt.legend()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'R0_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'R0_mean' ] - agents_norm_grouped[ 'R0_ci' ] ), ( agents_norm_grouped[ 'R0_mean' ] + agents_norm_grouped[ 'R0_ci' ] ), alpha = .1 )

plt.title( 'Tasa de ataque' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'R0' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Rt_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Rt_mean' ] - agents_norm_grouped[ 'Rt_ci' ] ), ( agents_norm_grouped[ 'Rt_mean' ] + agents_norm_grouped[ 'Rt_ci' ] ), alpha = .1 )

plt.title( 'Tasa de reproducción afectiva' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Rt' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Cuarentena_mean' ] )

plt.title( 'Cuarentena' )
plt.xlabel( 'Fecha' )
plt.ylabel( '% cuarentena' )
plt.grid( True )

In [ ]:
agents_norm.tail()

In [ ]:
agents_norm[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ].to_csv( './data/agents/clean/{}-{}.csv'.format( city, scenario ), index = False )